In [1]:
import torch
import torch.nn as nn
from torch.func import vmap, grad, functional_call  # Nouvelle API PyTorch 2.x
import vutils

# Modèle simple
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(50, 10)
        self.linear2 = nn.Linear(10, 1)

    def forward(self, x):
        output = self.linear(x)
        output = torch.relu(output)
        output = self.linear2(output)
        return output

# Initialisation
model = SimpleModel()
criterion = nn.MSELoss()

# Génération de données
batch_size = 50
x = torch.randn(batch_size, 50)  # Pas besoin de requires_grad=True ici
y = torch.randn(batch_size, 1)

grad_mean, grad_std = vutils.get_grad_mean_std(model, criterion, x, y)
grad_mean, grad_std

({0: tensor([[-6.4106e-03, -3.3353e-02,  1.3364e-02, -2.8887e-02, -7.5321e-02,
           -1.8185e-02, -3.2569e-03,  6.5186e-02, -3.2058e-02, -4.6944e-02,
            1.4342e-02, -2.3148e-02, -2.8157e-02, -2.4692e-02,  1.3931e-02,
           -2.7324e-02,  5.2846e-02, -1.5191e-03,  7.4774e-03, -3.4267e-02,
            1.7508e-02,  3.4185e-02,  2.7616e-02,  1.6929e-02,  2.0077e-02,
            6.9990e-02,  3.9333e-02, -5.1106e-02, -2.5873e-04, -4.9957e-02,
            1.9400e-02,  1.0376e-02, -1.2758e-02,  3.1844e-02,  2.8020e-02,
           -1.0619e-02, -3.3922e-02,  2.5086e-02,  1.7875e-02,  1.1833e-02,
           -1.9072e-02, -1.6390e-03, -3.5173e-02, -2.1596e-02, -3.2550e-02,
            3.3605e-02, -8.1000e-03,  1.7553e-02, -1.6241e-02,  1.3964e-02],
          [-4.5291e-03,  3.9730e-03,  6.2901e-03, -7.0890e-03, -6.1271e-03,
            8.0366e-03,  1.0265e-02,  1.2996e-02, -8.9413e-03, -1.4299e-02,
           -2.9414e-03, -1.4199e-03, -5.7256e-03, -3.2192e-03, -4.6267e-03,
        

In [9]:
list(model.parameters())


[Parameter containing:
 tensor([[-1.3782e-01, -6.0526e-02, -4.8069e-02, -8.9524e-03, -5.4907e-02,
          -3.6239e-02,  4.3580e-02,  1.3878e-01, -8.8553e-02, -3.5872e-02,
           1.3221e-01, -5.6092e-02,  1.9604e-02, -2.4736e-02,  1.3272e-01,
          -1.1607e-01, -6.8721e-02,  4.4338e-02,  8.3945e-02, -7.4793e-02,
           3.4903e-03,  4.3187e-02, -1.2079e-01, -7.7440e-02,  4.4152e-02,
           2.3509e-02, -1.0554e-01, -2.2795e-02, -2.3290e-02,  3.6980e-02,
           1.5180e-02, -5.4494e-02, -3.4270e-02,  1.2983e-01, -5.0781e-02,
           4.6702e-02,  7.4224e-02,  7.4471e-02,  1.3889e-01, -6.9914e-02,
           9.3390e-02, -1.5967e-02,  1.1818e-01,  4.0784e-03,  7.5586e-02,
          -1.0124e-01, -7.0380e-02,  1.0815e-01,  1.3227e-01,  1.0825e-01],
         [-2.6620e-03, -1.3538e-01,  3.2169e-02,  1.1253e-01, -1.1094e-01,
          -1.0437e-01,  7.8708e-02,  6.4943e-02, -7.0617e-02,  5.3161e-02,
           5.8821e-02,  4.6184e-02, -9.1995e-02, -7.2641e-02,  6.8709e-02,
 